In [1]:
import requests
import pandas as pd
from datetime import datetime

# Helper function to calculate age
def calculate_age(birthdate):
    today = datetime.today()
    return today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))

# Inputs
league_id = '1052985975103762432'  # Replace with your Sleeper League ID
weeks = range(1, 14)  # Adjust for the number of weeks in the season
base_url = f'https://api.sleeper.app/v1/league/{league_id}'

# Fetch rosters (teams)
rosters_response = requests.get(f'{base_url}/rosters')
rosters_data = rosters_response.json()

# Fetch users (owners)
users_response = requests.get(f'{base_url}/users')
users_data = users_response.json()

# Fetch player details for age calculation
players_response = requests.get('https://api.sleeper.app/v1/players/nfl')
players_data = players_response.json()

# Map owner IDs to usernames
owner_to_username = {user['user_id']: user['display_name'] for user in users_data}

# Prepare the data
all_data = []

for week in weeks:
    # Fetch weekly matchups (points scored)
    matchups_response = requests.get(f'{base_url}/matchups/{week}')
    matchups_data = matchups_response.json()
    
    # Process data for each team
    for roster in rosters_data:
        team_id = roster['roster_id']
        owner_id = roster.get('owner_id', 'Unknown')
        username = owner_to_username.get(owner_id, f'Unknown_User_{team_id}')
        team_name = f"Team {team_id} ({username})"
        
        # Get players in the roster
        player_ids = roster.get('players', [])
        player_ages = []
        for player_id in player_ids:
            player_info = players_data.get(player_id, {})
            birth_date_str = player_info.get('birth_date')
            if birth_date_str:
                birth_date = datetime.strptime(birth_date_str, '%Y-%m-%d')
                player_ages.append(calculate_age(birth_date))
        
        # Calculate average age of players
        average_age = sum(player_ages) / len(player_ages) if player_ages else None
        
        # Get points scored for the team this week
        weekly_points = next((m['points'] for m in matchups_data if m['roster_id'] == team_id), 0)
        
        # Append data
        all_data.append({
            'week': week,
            'team': team_name,
            'username': username,
            'points_scored': weekly_points,
            'average_age': average_age
        })

# Create a DataFrame
df = pd.DataFrame(all_data)

# Pivot the DataFrame for a wide format
df_points = df.pivot(index='username', columns='week', values='points_scored').add_prefix('points_week_')
df_age = df.pivot(index='username', columns='week', values='average_age').add_prefix('age_week_')

# Merge the wide-format DataFrames
df_wide = pd.concat([df_points, df_age], axis=1)

# Reset index for readability
df_wide.reset_index(inplace=True)

# Display the DataFrame
print(df_wide)


week         username  points_week_1  points_week_2  points_week_3  \
0         AlexFrame59         130.40         111.12         125.92   
1           Clayboski         100.72         159.00         150.66   
2           Drewski98          77.44          92.56         141.76   
3                Hege          63.78         120.80         131.94   
4          MattiICE23         119.78         108.80         126.86   
5            ShepC130          86.46         113.82          95.40   
6         TheHumanEgg         123.60         114.28         110.90   
7          Willythicc         132.56         120.40         164.78   
8            andrewi5         128.42         168.52         119.26   
9             bheck80         129.64         137.16         100.58   
10    garrettrsimpson         106.68          98.22         127.00   
11          jwoods329         150.80          82.98         122.62   

week  points_week_4  points_week_5  points_week_6  points_week_7  \
0            134.62  

In [2]:
df_wide.columns

Index(['username', 'points_week_1', 'points_week_2', 'points_week_3',
       'points_week_4', 'points_week_5', 'points_week_6', 'points_week_7',
       'points_week_8', 'points_week_9', 'points_week_10', 'points_week_11',
       'points_week_12', 'points_week_13', 'age_week_1', 'age_week_2',
       'age_week_3', 'age_week_4', 'age_week_5', 'age_week_6', 'age_week_7',
       'age_week_8', 'age_week_9', 'age_week_10', 'age_week_11', 'age_week_12',
       'age_week_13'],
      dtype='object', name='week')

In [3]:
#!pip install pymysql sqlalchemy pandas
from sqlalchemy import create_engine
from sqlalchemy.types import Float, String

# MySQL Connection Details
user = 'public'
password = 'dse511project'
host = '9QQ7GY3'
database = 'team_data'


# Create MySQL Engine
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

df_wide.to_sql('sleeperdata', engine, if_exists='replace', index=False)
print("DataFrame saved to MySQL!")

DataFrame saved to MySQL!
